# 1. Script set-up

In [261]:
#Create Folium map
import folium as fol
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

# 2. Import data

In [262]:
#import tunnel data
df_tun = pd.read_csv('tunnels_list.csv')
df_tun.head(5)

,TunnelName,WikiLink,TunnelType,Length[m],OpeningDate,Latitude,Longitude
0,Ampthill Tunnel,/wiki/Ampthill_Tunnel,Railway,654.0,1859,52.033611,-0.513333
1,Linslade Tunnel,/wiki/Linslade_Tunnel,Railway,249.0,"1837, 1859, 1876",51.925278,-0.676389
2,Old Warden Tunnel,/wiki/Old_Warden_Tunnel,Railway,807.0,1853,52.088333,-0.375833
3,Wymington Tunnel,No link,Railway,1690.0,1859,NaN,NaN
4,Clifton Down Tunnel,/wiki/Clifton_Down_Tunnel,Railway,1601.0,1874,51.469444,-2.618889


In [263]:
#import traffic data
df_traf = pd.read_csv('traffic_data.csv')
df_traf.head(5)

,latitude,longitude,link_length_km,pedal_cycles,two_wheeled_motor_vehicles,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,all_motor_vehicles,road_name
0,51.945910,-0.211865,4.2,0.0,211.5,34685.0,78.0,6508.5,2413.0,43896.5,A1(M)
1,51.981648,-0.196827,5.3,0.0,117.5,19698.0,22.0,3740.0,1893.5,25471.0,A1(M)
2,52.010865,-0.203506,0.4,0.0,81.0,13547.0,15.0,2565.5,1301.0,17509.5,A1(M)
3,52.014151,-0.205502,0.4,0.0,81.0,13547.0,15.0,2565.5,1301.0,17509.5,A1(M)
4,51.496059,-0.082578,0.4,674.0,815.0,7937.5,410.5,3070.5,308.5,12542.0,A100


# 3. Create HTML map

## 3.1. Base map

In [264]:
# Base map
map = fol.Map(location=[51.5,0], zoom_start=5, control_scale=True)

## 3.2. Create layer - UK districts

In [265]:
#Create Geojson map layer of UK districts
fg_districts = fol.FeatureGroup(name="UK districts")
fg_districts.add_child(fol.GeoJson(open('Local_Authority_Districts_December_2016_Ultra_Generalised_Clipped_Boundaries_in_Great_Britain.geojson').read(),
name='geojson'))

## 3.3. Create layer - Non road/rail tunnels

In [266]:
#Creating feature group of all non road/rail tunnels
fg_other_tunnels=fol.FeatureGroup("All other tunnels" , show=True)
for index, row in df_tun.iterrows():
    
    lat = row['Latitude']
    lon = row['Longitude']
    name = row['TunnelName']
    length = row['Length[m]']
    tunneltype = row['TunnelType']
    opendate = row['OpeningDate']

    if 'Road' not in tunneltype and 'Rail' not in tunneltype:    
        if abs(lat) > 0 and abs(lon) >0:
            fg_other_tunnels.add_child(fol.CircleMarker(location=[lat,lon], tooltip=name,
            popup='Length: ' + str(length) + 'm<br>Tunnel type: ' + tunneltype + '<br>Opened: ' +
                opendate, radius=0.25*math.sqrt(length), fill_color='blue', color='black',fill_opacity=0.7,icon='info-sign'))

## 3.4. Create layer - Road tunnels

In [267]:
#Creating feature group of road tunnels
fg_road_tunnels=fol.FeatureGroup("Road tunnels" , show=True)
for index, row in df_tun.iterrows():
    
    lat = row['Latitude']
    lon = row['Longitude']
    name = row['TunnelName']
    length = row['Length[m]']
    tunneltype = row['TunnelType']
    opendate = row['OpeningDate']
    
    if 'Road' in tunneltype:    
        if abs(lat) > 0 and abs(lon) >0:
            fg_road_tunnels.add_child(fol.CircleMarker(location=[lat,lon], tooltip=name,
            popup='Length: ' + str(length) + 'm<br>Tunnel type: ' + tunneltype + '<br>Opened: ' +
                opendate, radius=0.25*math.sqrt(length), fill_color='red', color='black',fill_opacity=0.7,icon='info-sign'))

## 3.5. Create layer - Rail tunnels

In [268]:
#Creating feature group of road tunnels
fg_rail_tunnels=fol.FeatureGroup("Railway tunnels" , show=True)
for index, row in df_tun.iterrows():
    
    lat = row['Latitude']
    lon = row['Longitude']
    name = row['TunnelName']
    length = row['Length[m]']
    tunneltype = row['TunnelType']
    opendate = row['OpeningDate']
    
    if 'Rail' in tunneltype:    
        if abs(lat) > 0 and abs(lon) >0:
            fg_rail_tunnels.add_child(fol.CircleMarker(location=[lat,lon], tooltip=name,
            popup='Length: ' + str(length) + 'm<br>Tunnel type: ' + tunneltype + '<br>Opened: ' +
                opendate, radius=0.25*math.sqrt(length), fill_color='yellow', color='black',fill_opacity=0.7,icon='info-sign'))

## 3.6. Create layer - Traffic

In [269]:
def traffic_colour(value):
    if int(value) > 15000:
        return 'red'
    elif int(value) > 6000:
        return 'orange'
    else:
        return 'green'

In [270]:
# #Creating feature group of road tunnels
# fg_traffic=fol.FeatureGroup("Traffic")
# for index, row in df_traf.iterrows():
    
#     lat = row['latitude']
#     lon = row['longitude']
#     name = row['road_name']
#     length = row['link_length_km']
#     allmotors = row['all_motor_vehicles']
#     cyclists = row['pedal_cycles']
#     motorbikes = row['two_wheeled_motor_vehicles']
#     cars = row['cars_and_taxis']
#     buses = row['buses_and_coaches']
#     lgvs = row['lgvs']
#     hgvs = row['all_hgvs']
    
#     if abs(lat) > 0 and abs(lon) >0:
#         fg_traffic.add_child(fol.Marker(location=[lat,lon], tooltip=name,
#         popup='Length: ' + str(length) + 'km<br><br>Average annual daily flows<br>Total vehicles: ' + str(allmotors) 
#             + '<br>No. of cyclists: ' + str(cyclists) + '<br>No. of motorbikes: ' + str(motorbikes) 
#             + '<br>No. of cars/taxis: ' + str(cars) + '<br>No. of buses/coaches: ' + str(buses) + '<br>No. of lgvs: ' 
#             + str(lgvs) + '<br>No. of hgvs: ' + str(hgvs),
#             icon=fol.Icon(color=traffic_colour(allmotors), icon='info-sign')))

In [271]:
#Creating feature group of road tunnels
fg_traffic=fol.FeatureGroup("Traffic (road tunnels)" , show=False)
for index, row in df_traf.iterrows():
    
    lat = row['latitude']
    lon = row['longitude']
    name = row['road_name']
    length = row['link_length_km']
    allmotors = row['all_motor_vehicles']
    cyclists = row['pedal_cycles']
    motorbikes = row['two_wheeled_motor_vehicles']
    cars = row['cars_and_taxis']
    buses = row['buses_and_coaches']
    lgvs = row['lgvs']
    hgvs = row['all_hgvs']
    
    if abs(lat) > 0 and abs(lon) >0:
        fg_traffic.add_child(fol.CircleMarker(location=[lat,lon], tooltip=name,
        popup='Length: ' + str(length) + 'km<br><br>Average annual daily flows<br>Total vehicles: ' + str(allmotors) 
            + '<br>No. of cyclists: ' + str(cyclists) + '<br>No. of motorbikes: ' + str(motorbikes) 
            + '<br>No. of cars/taxis: ' + str(cars) + '<br>No. of buses/coaches: ' + str(buses) + '<br>No. of lgvs: ' 
            + str(lgvs) + '<br>No. of hgvs: ' + str(hgvs),
        radius=5, fill_color=traffic_colour(allmotors), color='black',fill_opacity=0.9,icon='info-sign'))

## 3.6. Add layers to base map

In [272]:
#Add feature groups onto map and save as html
map.add_child(fg_districts)
map.add_child(fg_traffic)
map.add_child(fg_road_tunnels)
map.add_child(fg_rail_tunnels)
map.add_child(fg_other_tunnels)

# #Turn off traffic as default
# fol.Layer(fg_traffic,show=False)

#Add layer control
fol.LayerControl(collapsed=False).add_to(map)

# 4. Publish output

In [273]:
map.save("map.html")